In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from cascid.database import get_db

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pickle as pk

In [22]:
db = get_db()
db

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,PAT_1708,3156,NaN,NaN,NaN,NaN,73,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1708_3156_175.png,False
2294,PAT_46,880,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,12.0,BCC,True,True,False,True,False,False,PAT_46_880_140.png,True
2295,PAT_1343,1217,NaN,NaN,NaN,NaN,74,NaN,NaN,NaN,...,NaN,SEK,False,False,False,False,False,False,PAT_1343_1217_404.png,False
2296,PAT_326,690,False,False,POMERANIA,POMERANIA,58,True,FEMALE,True,...,4.0,BCC,True,False,False,False,False,True,PAT_326_690_823.png,True


In [23]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2298 entries, 0 to 2297
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   patient_id           2298 non-null   object 
 1   lesion_id            2298 non-null   int64  
 2   smoke                1494 non-null   object 
 3   drink                1494 non-null   object 
 4   background_father    1480 non-null   object 
 5   background_mother    1476 non-null   object 
 6   age                  2298 non-null   int64  
 7   pesticide            1494 non-null   object 
 8   gender               1494 non-null   object 
 9   skin_cancer_history  1494 non-null   object 
 10  cancer_history       1494 non-null   object 
 11  has_piped_water      1494 non-null   object 
 12  has_sewage_system    1494 non-null   object 
 13  fitspatrick          1494 non-null   float64
 14  region               2298 non-null   object 
 15  diameter_1           1494 non-null   f

In [24]:
df_filtered = db[["patient_id", "lesion_id", "diameter_1", "diameter_2", "diagnostic", "fitspatrick", "img_id"]]
df_filtered

,patient_id,lesion_id,diameter_1,diameter_2,diagnostic,fitspatrick,img_id
0,PAT_1516,1765,NaN,NaN,NEV,NaN,PAT_1516_1765_530.png
1,PAT_46,881,6.0,5.0,BCC,3.0,PAT_46_881_939.png
2,PAT_1545,1867,NaN,NaN,ACK,NaN,PAT_1545_1867_547.png
3,PAT_1989,4061,NaN,NaN,ACK,NaN,PAT_1989_4061_934.png
4,PAT_684,1302,5.0,5.0,BCC,1.0,PAT_684_1302_588.png
...,...,...,...,...,...,...,...
2293,PAT_1708,3156,NaN,NaN,ACK,NaN,PAT_1708_3156_175.png
2294,PAT_46,880,13.0,12.0,BCC,3.0,PAT_46_880_140.png
2295,PAT_1343,1217,NaN,NaN,SEK,NaN,PAT_1343_1217_404.png
2296,PAT_326,690,5.0,4.0,BCC,3.0,PAT_326_690_823.png


In [25]:
df_filtered_null = df_filtered[df_filtered.isna().any(axis=1)] # Select rows where there is any null value
df_filtered_not_null = df_filtered[~(df_filtered.isna().any(axis=1))] # Select rows where there is any null value
df_filtered_null, df_filtered_not_null

(     patient_id  lesion_id  diameter_1  diameter_2 diagnostic  fitspatrick  \
 0      PAT_1516       1765         NaN         NaN        NEV          NaN   
 2      PAT_1545       1867         NaN         NaN        ACK          NaN   
 3      PAT_1989       4061         NaN         NaN        ACK          NaN   
 5      PAT_1549       1882         NaN         NaN        SEK          NaN   
 8      PAT_1995       4080         NaN         NaN        ACK          NaN   
 ...         ...        ...         ...         ...        ...          ...   
 2288   PAT_1407       1402         NaN         NaN        ACK          NaN   
 2290   PAT_1294       1031         NaN         NaN        ACK          NaN   
 2293   PAT_1708       3156         NaN         NaN        ACK          NaN   
 2295   PAT_1343       1217         NaN         NaN        SEK          NaN   
 2297   PAT_1714       3189         NaN         NaN        SEK          NaN   
 
                      img_id  
 0     PAT_1516_176

In [26]:
df_filtered_not_null["diagnostic"].value_counts()

BCC    845
ACK    283
SCC    192
NEV     75
MEL     52
SEK     47
Name: diagnostic, dtype: int64

X_train.shape=(1120, 1)
X_test.shape=(374, 1)
y_train.shape=(1120,)
y_test.shape=(374,)


In [41]:
# Checking sklearn class distribution

# Split
_, _, y_train_split_test, y_test_split_test = train_test_split(df_filtered_not_null[["img_id"]].to_numpy(), df_filtered_not_null["diagnostic"].to_numpy())

# Count
train_distribution = pd.Series(y_train_split_test).value_counts(normalize=True)
test_distribution = pd.Series(y_test_split_test).value_counts(normalize=True)
original_distribution = df_filtered_not_null["diagnostic"].value_counts(normalize=True)

# Show dataframe
print("Relative class frequency (0-1):")
pd.DataFrame({
    "train":train_distribution,
    "test": test_distribution,
    "original": original_distribution
})

Relative class frequency (0-1):


,train,test,original
BCC,0.560714,0.580214,0.565596
ACK,0.192857,0.179144,0.189424
SCC,0.125000,0.139037,0.128514
NEV,0.050893,0.048128,0.050201
MEL,0.035714,0.032086,0.034806
SEK,0.034821,0.021390,0.031459
